In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from water_pollution.api import ann_utils as util

2021-12-02 09:16:07.590610: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-02 09:16:07.590682: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
stationsdf = util.get_stations_df()

In [5]:
stationsdf

,label,alt,river_id,river_label,coord,mean_nitrate
id,,,,,,
6000990,BELRUPT,292,U---0000,La Saône,"(48.090273327987276, 6.101940890815991)",4.513699
6000998,MONTHUREUX-SUR-SAONE 2,241,U---0000,La Saône,"(48.01825125838868, 5.941492053405311)",6.428571
6000993,JONVELLE,230,U---0000,La Saône,"(47.93638044949666, 5.923536232871648)",9.433333
6001000,CENDRECOURT,213,U---0000,La Saône,"(47.84026083741255, 5.917378095285407)",9.233333
6003600,SCEY-SUR-SAONE-ET-ST-ALBI,210,U---0000,La Saône,"(47.661731120409826, 5.972459738821173)",8.421519
6002500,PORT-SUR-SAONE,208,U---0000,La Saône,"(47.69107833426886, 6.039291564176292)",7.489474
6005500,APREMONT 1,190,U---0000,La Saône,"(47.39571566626848, 5.544088676925676)",10.553020
6011000,AUXONNE 1,185,U---0000,La Saône,"(47.194519280308995, 5.382571963766932)",10.828667
6017050,CHARREY-SUR-SAONE,180,U---0000,La Saône,"(47.07369860831355, 5.165636185828038)",10.831410


In [6]:
from datetime import date
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

In [7]:
(stationsdf.index)

Int64Index([6000990, 6000998, 6000993, 6001000, 6003600, 6002500, 6005500,
            6011000, 6017050, 6017070, 6039500, 6037400, 6053800, 6045800,
            6810010, 6059500],
           dtype='int64', name='id')

In [8]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  # Loads env variable from the root .env file
SAV_PATH = os.getenv('API_SAV_DIR')
SAV_PATH

'api_data/'

In [10]:
for station_id in stationsdf.index :

    # Build the cache filename
    stamp = date.today().strftime('%Y%m%d')
    cachefile_name = stamp + str(station_id) + '.cache'
    cachefile_path = SAV_PATH + 'cache/' + cachefile_name

    # If cache file exists, loads from it
    # Else, builds the prediction from WeatherAPI
    if os.path.isfile(cachefile_path):
        predicteddf = pd.read_pickle(cachefile_path)
        print('from cache')
    else :
        predicteddf = util.get_station_weather_prediction_df(station_id)
        predicteddf.to_pickle(cachefile_path)
        print('from API')


from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache
from cache


In [40]:
def plot_it(station_id,preddf):
    
    with plt.style.context('fivethirtyeight'):
    
        # Used Variables
        today = date.today()
        rmse = 2.19
        station_name = str(station_id)

        plt.figure(figsize=(12,5))
        ax = plt.gca()

        # Date Format
        dt_fmt = mdates.DateFormatter('%y-%m-%d')
        ax.xaxis.set_major_formatter(dt_fmt)

        # Prediction Plot
        plt.title(f'"{station_name}"\n Nitrate Pollution Prediction')
        plt.plot(preddf.index,preddf.prediction)
        plt.ylabel('Concentration (mg/L)')


        # Plots the rmse delta
        plt.fill_between(preddf.index,
                         preddf.prediction - rmse,
                         preddf.prediction + rmse,
                         color='k',alpha=.05)

        # Plots the today line
        ax.axvline(x=today, ymin=0., ymax=0.9, c='red', alpha=0.6, linewidth=2)
        ax.text(today-timedelta(1), 0,"Today", alpha=1,ha='center',color='red',rotation=90) # centré

        plt.xticks(rotation=25)

        plt.ylim(bottom=-1)
        plt.ylim(top=plt.ylim()[1]+3)
        plt.show()